# Assignment Applied Machine Learning BSc IK 

## Notebook made by

**Gebruik graag dit formaat**

* Voor de namen:  voornaam rest van je naam, voornaam rest van je naam,....
* je studentnummers: hetzelfde: scheidt met `,`
* je emails: hetzelfde: scheidt met `,`
* voor je groep: **alleen de hoofdletter** (iets als  `A` of `B` dus)

__Namen__:Anoniem
__Emails__:Anoniem
__Student id__:Anoniem
__Groep__:Anoniem

## Toelichting

* Een aantal opgaven worden automatisch nagekeken. Bij vrijwel alle opdrachten staan er een paar tests onder de opdracht, dit is voornamelijk om te zorgen dat je de juiste type output geeft. Dit zijn dus *NIET* alle tests, die komen er bij het graden nog bij.
* Elke vraag is 1 punt waard, tenzij anders aangegeven. Soms is die punt onderverdeeld in deelpunten, maar niet altijd. 

## Voor het inleveren!

* Pas niet de cellen aan, vooral niet die je niet kunt editen. Dit levert problemen op bij nakijken. Twijfel je of je per ongeluk iets hebt gewijzigd, kopieer dan bij inleveren je antwoorden naar een nieuw bestand, zodat het niet fout kan gaan.

* Zorg dat de code goed runt van boven naar beneden, verifieer dat door boven in Kernel -> Restart & Run All uit te voeren

## Na het inleveren!

* Het gebeurt erg vaak dat mensen een "leeg bestand" inleveren. Vaak een andere versie van de opgave die nog ergens op je computer rondslingerde. Zonde van al je werk toch!
* Dus, lever **minstens een half uur voor tijd in**. Download dan wat je hebt ingeleverd op Canvas. Geef het een andere naam om verwarring te voorkomen. En draai alle cellen, en bekijk het. Geen syntax fouten? Alle vragen gemaakt? Dan zit het vast wel goed, en hoef je niet in de zenuwen te zitten.

# Applied Machine Learning W5

In this week's assignment we will be zooming in on one of the cornerstones of Machine Learning: Gradient Descent. We have set up a few exercises that should get you familiar with how gradient descent works and how different loss functions effect the outcome of the algorithm. The rest of the exercises will be focussed on getting more familiar with linear regression. The asssignment consists of 4 subquestions, for a total number of 13 points.


## Subquestions
- 1.1 [Gradient Descent](#grad1) <br>
- 1.2 [Gradient Descent on the Tips dataset](#grad2) <br>
- 2. [Linear Regression](#linreg1) <br>
   - 2.1.[The Titanic Dataset and Scaling](#linreg_intro)
   - 2.3 [Cross Validation](#crossval) <br>
   - 2.4 [Grid Search](#gridsearch)

In [ ]:
import os
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
from typing import Tuple
from scipy.stats import mode
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# testing
from numpy.testing import *
from pandas.testing import assert_frame_equal
from nose.tools import assert_count_equal, assert_equal, assert_almost_equals

# Please do not remove this: 
np.random.seed(31415)

<a id="grad1" />

# Q1  Gradient descent

Do the exercise in the one but last slide of this week, related to the function $f(x,y)$ given there.

![](../../data/Week5/gd1.png)
1. Find the minimum analytically, by putting the two partial derivatives to 0 and solve for $x$ and $y$. You find a lot of possible value pairs, but most are spurious. Explain why they are spurious, and give the real answer(s).
2. Find the minimum using the gradient descent algorithm, using these two partial derivatives. In the slides you find the update step. Program it.  Add a while loop in whichn you check that either you have reached your maximum number of allowed steps (a hyperparameter of course), or $x$ and $y$ make the function go up after following the gradient (i.e., you are in a minumum). 
3. Experiment with the learning rate, and tabulate the learning rate against the number of steps, and explain what is going on. Also make the learning rate really large so you jump over the minimum. Not only give a nice table (or graphic, also explain what you have learned from about the size of the learning rate.

First find the minimum analytically, and try to explain how you did this, and how you removed the spurious answers.

### dx

1. ~~x = 0~~
2. ~~y = 0~~
3. x = 4
4. ~~y = 6~~

1 & 2 will always result in 0 for the derivative, even when constants are completely different, and can therefore be ignored (spurious answers).

For answer 3:
- 3x - 12 = 0
- 3x = 12
- x = 4

For answer 4:
- y - 6 = 0
- y = 6

Answer 4 can be ignored; this is de df / dx formula.

### dy

1. ~~x = 0~~
2. ~~y = 0~~
3. ~~x = 6~~
4. y = 4

1 & 2 will always result in 0 for the derivative, even when constants are completely different, and can therefore be ignored (spurious answers).

For answer 3:
- x - 6 = 0
- x = 6

Answer 3 can be ignored; this is de df / dy formula.

For answer 4:
- 3y - 12 = 0
- 3y = 12
- y = 4

### Answers

- x = 4
- y = 4

We will start by implementing the function `f` itself, and two two functions that take the partial derivatives with respect to x and y, in `df_dx` and `df_dy` respectively.

In [ ]:
# First, let us define our function and its partial derivatives
# Fill in the functions below

def f(x,y) -> float:
    '''
    Function that implements the function f from above, returning a float
    '''
    return 85 - 1 / 90 * x ** 2 * (x - 6) * y ** 2 * (y - 6)

def df_dx(x, y) -> float:
    '''
    Function that returns the partial derivative of f with respect to x, returning a float
    '''
    return -1 / 90 * x * (3 * x - 12) * y ** 2 * (y - 6)

def df_dy(x, y) -> float:
    '''
    Function that returns the partial derivative of f with respect to y, returning a float
    '''
    return -1 / 90 * x ** 2 * (x - 6) * y * (3 * y - 12)

In [ ]:
assert_equal(type(f(1.5, 3.0)), float)
assert_equal(type(df_dx(1.5, 3.0)), float)
assert_equal(type(df_dy(1.5, 3.0)), float)

After we have implemented these partial derivatives we will now implement the gradient descent algorithm that will, given a learning rate and a maximum number of steps compute the optimal values for x and y. Fill in the function below.

In [ ]:
# To help you along, first implement one step of the gradient descent algorithm in the function `gradient_step` below
# using your df_dx and df_dy functions. The function should return a tuple with the new values for x and y
def gradient_step(x, y, learning_rate) -> Tuple[float, float]:
    x = x - learning_rate * df_dx(x, y)
    y = y - learning_rate * df_dy(x, y)
    return (x, y)
    

In [ ]:
assert_equal(type(gradient_step(0.6, 0.5, 0.01)), tuple)
assert_equal(type(gradient_step(0.6, 0.5, 0.01)[0]), float)
assert_equal(type(gradient_step(0.6, 0.5, 0.01)[1]), float)

Finally, implement the complete `gradient_descent` algorithm, where you use the function `gradient_step` that you have defined previously. As starting values for x and y, use 0.6 and 0.5 respectively.

In [ ]:
def gradient_descent(learning_rate, max_num_steps)->Tuple[float, float]:
    """
    This function implements the gradient descent algorithm for the function f
    defined above, given a float learning_rate and an integer max_num_steps.
    You can use the functions f, df_dx and df_dy that you created above.
    The function should return a tuple containing the optimal x value and the optimal
    y value.
    """
    steps = 0
    # we will give you some starting values for x and y
    x, y = 0.6, 0.5
    
    for _ in range(max_num_steps):
        x, y = gradient_step(x, y, learning_rate)
        
    return (x, y)

In [ ]:
assert_equal(type(gradient_descent(0.01, 5)), tuple)
assert_equal(type(gradient_descent(0.01, 5)[0]), float)
assert_equal(type(gradient_descent(0.01, 5)[1]), float)

In the cell below, make a nice table/ plot where you experiment with different learning rates and explain what is happening.

In [ ]:
learning_rates = [0.01, 0.05, 0.1, 0.2, 0.5]
max_num_steps = [5, 25, 50, 100, 500]

table = pd.DataFrame(index=learning_rates, columns=max_num_steps)

for lr in learning_rates:
    for mns in max_num_steps:
        table.loc[lr, mns] = np.round(gradient_descent(lr, mns), 2)

display(table)

"""
Learning steps 0.01 through 0.20 in the chosen list all arrive at (4.0, 4.0) for x, y.
A learning step of 0.50 is too big, overshoots and ends up unable to get to the correct answer.
The smaller learning steps need significantly more steps to get to the correct answer.
"""

<a id="grad2" />

# Q1b gradient descent on the Tips dataset

Now that we have implemented the gradient descent algorithm, we are going to actually use it in a real world dataset, using the `tips` dataset that we have seen before.

Run the code below, in which you plot the tip against the total bill. You can see the regression line, and you can also compute the best fitting coefficients using sklearn.

But now we ask you to find them yourself using gradient descent. So find the best intercept and slope.

### Plot

Plot on top of this scatterplot your found line. 

```
tips = sns.load_dataset('tips')
sns.regplot(x='total_bill', y='tip', data=tips);
```

To find the best intercept and slope using gradient descent, we need to define a cost function that measures how well our line fits the data. In this case, we can use the mean squared error (MSE) as our cost function:

cost(m, b) = (1/N) * sum((y_i - (m*x_i + b))^2)

or, as a nice Latex formula: $\frac{\sum\limits_{i}(y_{i} - (m*x_i + b))^{2}}{N}$

where:

m: slope b: intercept x_i: total_bill of the i-th observation y_i: tip of the i-th observation N: total number of observations

Of course, after you have done this yourself you can check with scikit learn that you got the right values for the parameters :)

HINT: in the previous exercise we used `x` and `y` as variables, but in this exercise these variables are `m` and `b`, so be careful that you don't mix up these definitions. As with the previous exercise, implement a maximum number of steps, and also make sure that the new values for m and b do not make the cost function inrease.

In [ ]:
tips = sns.load_dataset('tips')
tips['total_bill'] = (tips['total_bill'] - tips['total_bill'].mean()) / tips['total_bill'].std()

In [ ]:
# We load in the dataset
tips = sns.load_dataset('tips')

def cost_function(m, b, x, y):
    cost_sum = [y_i - (m * x_i + b) ** 2 for x_i, y_i in zip(x, y)]
    return sum(cost_sum) / len(x)


# Define partial derivatives of cost function
def partial_m(m, b, x, y):
    dm_sum = [-2 * x_i * (y_i - (m * x_i + b)) for x_i, y_i in zip(x, y)]
    return sum(dm_sum) / len(x)

def partial_b(m, b, x, y):
    db_sum = [-2 * (y_i - (m * x_i + b)) for x_i, y_i in zip(x, y)]
    return sum(db_sum) / len(x)


In [ ]:
x_test = tips['total_bill']
y_test = tips['tip']
m_test = 1
b_test = 2


In [ ]:
# Fit line using gradient descent
# Fill in the function below, which is similar to the function before, except that we are now working with different
# functions for the cost function and partial derivatives.

def gradient_descent_tips(x, y, learning_rate, max_num_steps):
    
    # Initialize slope and intercept
    m, b = 0.1, 0.6
    
    for _ in range(max_num_steps):
        new = m - learning_rate * partial_m(m, b, x, y), b - learning_rate * partial_b(m, b, x, y)
        m, b = new

    # Return best estimates of m and b
    return m, b
    
m, b = gradient_descent_tips(tips['total_bill'], tips['tip'], 0.001, 1000)

# Plot scatterplot with regression line and gradient descent line
sns.regplot(x='total_bill', y='tip', data=tips)
plt.plot(tips['total_bill'], m*tips['total_bill']+b, color='red')
plt.show()

In [ ]:
m, b = gradient_descent_tips(tips['total_bill'], tips['tip'], 0.001, 1000)
assert np.allclose(np.shape(m), 1)

<a id="linreg1" />

# Q2 Cost function for linear regression

* Recall the loss/cost function given for linear regression.
1.  Often you see it written as not the mean squared error, but half of the mean squared error. Explain in two-three sentences the difference and what the influence of taking the half is.
2. Again look at the `tips` dataset with one explanatory variable `total_bill`. Investigate the influence of the few "high bill" datapoints on the regression line. Think about the meaning of taking the square of the error. 
    * Your investigation will take the form a table, with for each of the thresholds we have defined the corresponding optimal coefficient `a` and `b`. It is also nice to plot the lines with these coefficients over the original data, to see the differences.
    * Also try to use what we learned on confidence intervals to estimate something on the significance on the observed differences in the regression lines.

HINT: You are allowed to use the `LinearRegression` function from sklearn for this exercise. Also, don't round the coefficients, just leave them like they come out of the `LinearRegressor` function. Check the documentation for the function to see how you can get the coefficient and intercept parameters that you want out of it.

YOUR ANSWER HERE

In [ ]:
tips = sns.load_dataset('tips')
sns.regplot(x='total_bill', y='tip', data=tips,  label='full fit');

# It is convenient to iterate over a list of thresholds that truncate the data
# You should perform Linear Regression on each of the truncated datasets and put your results in the 
# variable `threshold_table`, with the column names 'coefficient' and 'intercept'.

thresholds = [25, 30, 35, 40, 45]
threshold_table = pd.DataFrame()
threshold_table.index = thresholds
threshold_table['coefficient'] = 0.0
threshold_table['intercept'] = 0.0

for threshold in thresholds:
    threshold_tips = tips[tips['total_bill'] > threshold]
    model = LinearRegression()
    
    model.fit(threshold_tips[['total_bill']], threshold_tips['tip'])
    
    threshold_table.loc[threshold, 'coefficient'] = model.coef_[0]
    threshold_table.loc[threshold, 'intercept'] = model.intercept_

# If we want to plot this, we should define a range of x values to plot this for.
x = np.arange(0, 50)

sns.scatterplot(x="total_bill", y="tip", data=tips)
for threshold in thresholds:
    coefficient = threshold_table.loc[threshold, 'coefficient']
    intercept = threshold_table.loc[threshold, 'intercept']
    sns.lineplot(x=x, y=x*coefficient+intercept, label=threshold)

In [ ]:
assert_equal(threshold_table.shape, (5, 2))

Now that we have experimented a bit with the loss function for linear regression we are going to do some experiments on the (hopefully) familiar `titanic` dataset.

<a id="linreg_intro" />

# Titanic


## P1 standardization

* Load the titanic dataset.
* Impute the age variable, motivate your choice.
* Split the dataset into a train and test set, use a 70/30 split.
* Now use lineair regression to predict survived from the other numeric variables (visible in the `t.describe()` output). Output the regression weights, and compute Rsquare and RMSE. Just use all numeric columns here (there are 5) although the pclass variable is really a categorical variable.

For this question it is OK to use the `LinearRegressionModel` from scikit-learn and the `train_test_split` function as well.


In [ ]:
titanic=sns.load_dataset('titanic')
print(titanic.shape)
print(titanic.head())
titanic.describe()

In [ ]:
from sklearn.impute import SimpleImputer

titanic['age'] = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(titanic[['age']])

# Split the data into test and train datasets, select variables which we do not need to preprocess 
# Use linear regression to predict survived 

from sklearn.model_selection import train_test_split

X = titanic[['pclass', 'age', 'sibsp', 'parch', 'fare']]
y = titanic['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

lin_reg_model = LinearRegression().fit(X_train, y_train)
predictions = lin_reg_model.predict(X_test)
print("['pclass', 'age', 'sibsp', 'parch', 'fare']", lin_reg_model.coef_)

# Compute R-squared and RMSE
r2 = r2_score(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
#WRITE YOUR CODE HERE
print(f"R-squared: {r2}")
print(f"RMSE: {rmse}")

In [ ]:
assert X_train.shape == (623, 5)
assert X_test.shape == (268, 5)
assert y_train.shape == (623,)
assert y_test.shape == (268,)


* Now z-transform all explanatory variables and do the same. For this exercise, fit the scalar on the train data and use it to transform both the train and test sets.
* Describe what you observe and what you may learn from this exercise.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Use the StandardScaler class from scikit-learn to z-transform the explanatory variables:
X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit(X_train).transform(X_test)
lin_reg_model = LinearRegression().fit(X_train_scaled, y_train)
predictions_z = lin_reg_model.predict(X_test_scaled)
print("['pclass', 'age', 'sibsp', 'parch', 'fare']", lin_reg_model.coef_)

# Get predictions on the survived, then compute R-squared and RMSE
r2_scaled = r2_score(y_test, predictions_z)
rmse_scaled = mean_squared_error(y_test, predictions_z, squared=False)
#WRITE YOUR CODE HERE
print(f"R-squared (scaled): {r2_scaled}")
print(f"RMSE (scaled): {rmse_scaled}")

In [ ]:
assert X_train_scaled.shape == (623, 5)
assert X_test_scaled.shape == (268, 5)


<a id="crossval" />

For the next two questions, we are going to be working with our scaled data!

## P2 cross validation
Instead of dividing our dataset in a train and test set, we are going to do 5 fold cross validation with our scaled data, and report the values that we got for this.

* Now just take the whole dataset and implement 5 fold cross validation yourself using sklearn.
* Then do a linear regression 5 times on the obtained train-test fold splits.
* Plot the found evaluation values, and compute the mean and std.


In [ ]:
#We will use the KFold function from sklearn
from sklearn.model_selection import KFold


kf = KFold(n_splits=5, shuffle=True, random_state=42) # don't change the random state

#create lists to store scores for r2 and rsme, split the data as usual, predict survival 
#as before with linear regression, compute our r2 and rsme and append these to lists, plot in your own way.
r2_scores = []
rmse_scores = []

# Iterate over each fold
for train_index, test_index in kf.split(titanic):
    X_train_split, X_test_split = X.iloc[train_index], X.iloc[test_index]
    y_train_split, y_test_split = y.iloc[train_index], y.iloc[test_index]

    lin_reg_model = LinearRegression().fit(X_train_split, y_train_split)
    predictions_f = lin_reg_model.predict(X_test_split)

    r2_scores.append(r2_score(y_test_split, predictions_f))
    rmse_scores.append(mean_squared_error(y_test_split, predictions_f, squared=False))

# plot the scores
plt.plot(r2_scores, label='R-squared')
plt.plot(rmse_scores, label='RMSE')
plt.legend()
plt.show()

# mean and std
print(f"R-squared mean: {np.mean(r2_scores)}")
print(f"R-squared std: {np.std(r2_scores)}")
print(f"RMSE mean: {np.mean(rmse_scores)}")
print(f"RMSE std: {np.std(rmse_scores)}")

In [ ]:
assert np.allclose(np.shape(r2_scores), (5,))
assert np.allclose(np.shape(rmse_scores), (5,))


<a id="gridsearch" />

## P3 Grid search

* Split titanic into train, validation, test using a 60/20/20 split for train, validation and test respectively.
* Use the validation set to find optimal values for the regularization parameter, and the choice between lasso and ridge.
* You are not allowed to use the `GridSearchCV` function, as this does not really do what we want, so
we will be implementing it from scratch ourselves. We have already given you a list with three options for the regularization that you should use.
* Use the r2 score to measure how well a particular setting works, and save this in a dictionary `results` with as keys `lasso` and `ridge` and as values the values for the regularization parameter in the order given in the list of possible values.

HINT: sklearn does not have a function to directly split a dataset into train, validation and test, but you can use `train_test_split` twice, once to create separate train and test sets, and then once more to split the train set into a train and validation portion.



In [ ]:
# Use these functions for Lasso and Ridge
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import f1_score

# Split data into training, validation, and test sets (remember to scale features!)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
X_train_scaled = StandardScaler().fit_transform(X_train)
X_val_scaled = StandardScaler().fit_transform(X_val)
X_test_scaled = StandardScaler().fit(X_train).transform(X_test)

# Define hyperparameters for grid search
alpha_options = [0.1, 1.0, 10.0]

# Perform grid search using train to the normalized train to train a model
# reporting the validation r2 in the table, calculated on the validation set. 
# you can use the `r2_score` from sklearn to calculate the rqaured error
results = {'ridge': [], 'lasso': []}

for alpha_option in alpha_options:
    ridge_model = Ridge(alpha=alpha_option).fit(X_train_scaled, y_train)
    ridge_predictions = ridge_model.predict(X_val_scaled)
    results['ridge'].append(r2_score(y_val, ridge_predictions))

    lasso_model = Lasso(alpha=alpha_option).fit(X_train_scaled, y_train)
    lasso_predictions = lasso_model.predict(X_val_scaled)
    results['lasso'].append(r2_score(y_val, lasso_predictions))
    
scores_dataframe = pd.DataFrame(results)
scores_dataframe.index = alpha_options

In [ ]:
assert X_train.shape == (534, 5)
assert y_train.shape == (534,)
assert X_val.shape == (178, 5)
assert y_val.shape == (178,)
assert X_test.shape == (179, 5)
assert y_test.shape == (179,)



* After having done this on the validaion set, run all of these possibilities on the test set. 
* Think, display, and write a conclusion and what you have learned. 
* You can reuse most of the code from above to do this, 

In [ ]:
# Define hyperparameters for grid search
alpha_options = [0.1, 1.0, 10.0]

# Perform grid search using train to the normalized train to train a model
# reporting the test r2 in the table, calculated on the test set. 
# you can use the `r2_score` from sklearn to calculate the rqaured error
test_results = {'ridge': [], 'lasso': []}

for alpha_option in alpha_options:
    ridge_model = Ridge(alpha=alpha_option).fit(X_train_scaled, y_train)
    ridge_predictions = ridge_model.predict(X_test_scaled)
    test_results['ridge'].append(r2_score(y_test, ridge_predictions))

    lasso_model = Lasso(alpha=alpha_option).fit(X_train_scaled, y_train)
    lasso_predictions = lasso_model.predict(X_test_scaled)
    test_results['lasso'].append(r2_score(y_test, lasso_predictions))
test_scores_dataframe = pd.DataFrame(test_results)
test_scores_dataframe.index = alpha_options

In [ ]:
print("Validation scores")
print(scores_dataframe)
print("Test scores")
print(test_scores_dataframe)

# conclusion
# ridge is much better than lasso for this dataset
# the best alpha is 1.0 because it has the best r2 scores when taking into account both validation and test results